<a href="https://colab.research.google.com/github/Atharv24-Atreus/Cyber_Security-/blob/main/cyberLab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.2 MB/s eta 0:00:00


In [ ]:
from scapy.all import rdpcap, TCP, UDP, ICMP

# Path to the pcap file
pcap_file = "Sample.pcap"  # Change this to your actual file path

# Read the packets from the pcap file
packets = rdpcap(pcap_file)

# Initialize counters
tcp_count = 0
udp_count = 0
icmp_count = 0

# Iterate through packets and count protocol occurrences
for packet in packets:
    if packet.haslayer(TCP):
        tcp_count += 1
    elif packet.haslayer(UDP):
        udp_count += 1
    elif packet.haslayer(ICMP):
        icmp_count += 1

# Display the results
print(f"TCP Packets: {tcp_count}")
print(f"UDP Packets: {udp_count}")
print(f"ICMP Packets: {icmp_count}")

TCP Packets: 2093
UDP Packets: 393
ICMP Packets: 0


In [ ]:
from collections import Counter
from scapy.all import rdpcap, IP, TCP, ICMP

# Path to the pcap file (update as needed)
pcap_file_path = "Sample.pcap"

# Read packets from the pcap file
packets = rdpcap(pcap_file_path)

# Initialize data structures for analysis
ip_counter = Counter()
syn_flood_attempts = Counter()
icmp_requests = Counter()

# Analyze packets
for packet in packets:
    if packet.haslayer(IP):
        ip_src = packet[IP].src  # Extract source IP

        # Count occurrences of each IP (detect excessive requests)
        ip_counter[ip_src] += 1

        # Detect TCP SYN flood attempts (repeated SYNs from the same source)
        if packet.haslayer(TCP) and packet[TCP].flags == "S":
            syn_flood_attempts[ip_src] += 1

        # Detect ICMP flood (excessive ICMP requests)
        if packet.haslayer(ICMP):
            icmp_requests[ip_src] += 1

# Identify potential brute-force attack (IPs making excessive requests)
brute_force_ips = [ip for ip, count in ip_counter.items() if count > 100]

# Identify possible SYN flood attack sources
syn_flood_ips = [ip for ip, count in syn_flood_attempts.items() if count > 50]

# Identify ICMP flood attack sources
icmp_flood_ips = [ip for ip, count in icmp_requests.items() if count > 50]

# Display results
print("Potential Brute-Force Attack Sources:", brute_force_ips)
print("Potential SYN Flood Attack Sources:", syn_flood_ips)
print("Potential ICMP Flood Attack Sources:", icmp_flood_ips)


Potential Brute-Force Attack Sources: ['10.11.12.101', '80.85.159.236', '194.87.147.244', '124.217.255.96', '94.140.114.6']
Potential SYN Flood Attack Sources: []
Potential ICMP Flood Attack Sources: []


In [ ]:
from scapy.all import rdpcap, DNSQR
from collections import Counter

# Path to the pcap file (update this to your actual file path)
pcap_file_path = "Sample.pcap"

# Read packets from the pcap file
packets = rdpcap(pcap_file_path)

# Extract DNS query names
dns_queries = []

for packet in packets:
    if packet.haslayer(DNSQR):  # Check if packet contains a DNS query
        dns_queries.append(packet[DNSQR].qname.decode())  # Extract domain name

# Count the most frequently queried domains
dns_counts = Counter(dns_queries)

# Display the top queried domains
print("Most Frequently Queried Domains:")
for domain, count in dns_counts.most_common(10):  # Show top 10
    print(f"{domain}: {count} times")


Most Frequently Queried Domains:
wpad.localdomain.: 68 times
wpad.: 44 times
isatap.localdomain.: 42 times
teredo.ipv6.microsoft.com.: 12 times
Smithers-PC.: 11 times
google.com.: 6 times
www.google.com.: 4 times
kh2714ldb.com.: 4 times
gmail.com.: 4 times
dns.msftncsi.com.: 2 times


In [4]:
from scapy.all import rdpcap, DNS, DNSQR
from collections import Counter

# Path to the pcap file (update this to your actual file path)
pcap_file_path = "Sample.pcap"

# Read packets from the pcap file
packets = rdpcap(pcap_file_path)

# Extract failed DNS query names
dns_failures = []

for packet in packets:
    if packet.haslayer(DNS) and packet.haslayer(DNSQR):  # Check if packet contains a DNS query
        dns_layer = packet[DNS]
        if dns_layer.qr == 1 and dns_layer.rcode != 0:  # Response packet with a failure code
            dns_failures.append(packet[DNSQR].qname.decode())

# Count the most frequently failed domains
dns_failure_counts = Counter(dns_failures)

# Display the top failed queries
print("Most Frequently Failed DNS Queries:")
for domain, count in dns_failure_counts.most_common(10):  # Show top 10 failures
    print(f"{domain}: {count} failures")


Most Frequently Failed DNS Queries:
teredo.ipv6.microsoft.com.: 6 failures
myip.opendns.com.localdomain.: 1 failures
